In [5]:
from utils.create_average_activation_vectors import model_setup
from classes.hook_manager import HookManager

In [4]:
model, tokenizer, device = model_setup("roneneldan/TinyStories-1M")

found device: cpu


In [ ]:



def get_activations(meta_data: dict, 
    loader: DataLoader, 
    tokenizer: AutoTokenizer, 
    device: str,
    model:AutoModelForCausalLM,
    label_map=None
    ) -> dict: 

    if label_map == None:
        label_map = loader.dataset.label_map
    res_stream_act_by_layer = dict()
    activation_ds_by_layer = {
        layer: ActivationDataset(label_map=label_map)
        for layer in range(meta_data["hidden_layers"])
    }

    if tokenizer.pad_token == None:
        tokenizer.pad_token = tokenizer.eos_token

    for ind, (text, label) in enumerate(tqdm(loader)):

        if ind > 5:
            break

        
        tokenized = tokenizer(
            text,
            padding=True,
            truncation=True,
            return_tensors='pt'
        ).to(device)

        with HookManager(model) as hook_manager:
            for layer in range(meta_data["hidden_layers"]):
                res_stream_act_by_layer[layer] = hook_manager.attach_residstream_hook(
                    layer=layer,
                    pre_mlp=False,
                    pythia=True if isinstance(model, GPTNeoXForCausalLM) else False
                )

            model(**tokenized)

        # flattening [batch, pad_size, ...] to [tokens, ...]
        attn_mask = tokenized.attention_mask.flatten() # [tokens]
        label = label.unsqueeze(-1).expand(-1, tokenized.attention_mask.shape[1]).flatten() # [tokens]

        for layer in range(meta_data["hidden_layers"]):
            res_stream_act_by_layer[layer] = res_stream_act_by_layer[layer][0].view(-1, meta_data["hidden_size"]) # [tokens, hidden_size]
            activation_ds_by_layer[layer].add_with_mask(res_stream_act_by_layer[layer], label, attn_mask)
    return activation_ds_by_layer
